#### 导入模块

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout,Flatten,Activation
from keras.layers import Conv2D,MaxPooling2D
from keras import backend as K 


### 定义变量

In [ ]:
batch_size = 32   # batch大小
num_classes = 10   # 类别个数
epochs = 200        # epoch次数
data_augmentation = True #什么意思？

### 下载数据

In [ ]:
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

In [ ]:
print(K.image_data_format())
print(K.image_dim_ordering())

### 处理数据（CNN 网络）

In [ ]:
## 是否需要？
if K.image_data_format() == 'channels_first':
    x_train = x_train.transpose((0,3,1,2))
    x_test = x_test.transpose((0,3,1,2))
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /=255
x_test /=255
print('x_train shape:',x_train.shape)
print(x_train.shape[0],'train samples')
print(x_test.shape[0],'test samples')
y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test,num_classes)

### 定义网络结构(CNN)

In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3),padding='same',input_shape=x_train.shape[1:]))  # input_shape中不包含batch大小
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),padding='same'))  # 只需要在第一层指定输入数据维数
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))    
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

### 网络编译

In [ ]:
opt = keras.optimizers.rmsprop(lr=0.0001,decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer=opt, metrics=['accuracy'])   # MLP

### 网络训练

In [ ]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train,y_train,
             batch_size=batch_size,
             epochs=epochs,
             validation_data=(x_test,y_test),
             shuffle=True)
else:
    print('Using real-time data augmentation.')
    datagen = ImageDataGenerator(
        featurewise_center=False, #set input mean to 0 over the dataset
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=0,
        width_shift_range=0,
        height_shift_range=0.1,
        horizontal_flip=True,
        vertical_flip=False)
    print(1)
    datagen.fit(x_train)

    model.fit_generator(datagen.flow(x_train,y_train,batch_size=batch_size),
                       steps_per_epoch=x_train.shape[0]//batch_size,
                       epochs=epochs,
                       validation_data=(x_test,y_test))

### 评估网络

In [ ]:
score = model.evaluate(x_test,y_test,verbose=0)
print('Test loss:',score[0])
print('Test accuracy',score[1])